In [ ]:
SVM_SKIP_TRAINING = False
MLP_SKIP_TRAINING = False

In [ ]:
import os
import sys

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

import src.paths as paths

sys.path.append(paths.SRC_DIR)
sys.path.append(paths.VGGISH_DIR)

%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.magic import register_line_cell_magic
from IPython import get_ipython

@register_line_cell_magic
def skip(line, cell=None):
    '''Skips execution of the current line/cell if line evaluates to True.'''
    if eval(line):
        return

    get_ipython().ex(cell)

# 1 - Extract features

In [ ]:
from src.feature_extract import extract_features_batch

# Define the batch size
BATCH_SIZE = 25  # You can adjust this based on your system's capabilities

batch_files = []
batch_out_files = []

for root, dirs, files in os.walk( os.path.join(paths.DATA_DIR, "openmic", "audio") ):
    for file in files:
        if file.endswith(".ogg"):
            ogg_file = os.path.join(root, file)

            file_base = os.path.splitext(file)[0]

            dir_ = os.path.join(paths.DATA_DIR, "features", file_base[:3])

            if not os.path.exists(dir_):
                os.makedirs(dir_)

            
            new_filename = os.path.join(dir_, f"{file_base}.tfrecord")

            # print(new_filename)
            
            if os.path.exists(new_filename):
                continue

            batch_files.append(ogg_file)
            batch_out_files.append(new_filename)

            # print(f"Adding {file_base}.ogg")


def chunks(lst1, lst2, chunk_size):
    """Yield successive chunk_size chunks from lst1 and lst2."""
    for i in range(0, min(len(lst1), len(lst2)), chunk_size):
        yield lst1[i:i + chunk_size], lst2[i:i + chunk_size]


assert len(batch_files) == len(batch_out_files)

print(f"About to process {len(batch_files)} files...")

# Initialize the progress bar, dark mode
pbar = tqdm(total=len(batch_files), desc="Extracting features", unit=" file")

files_processed = 0

for in_files, out_files in chunks(batch_files, batch_out_files, BATCH_SIZE):
    assert len(in_files) == len(out_files)

    count = len(in_files)

    extract_features_batch(in_files, out_files)

    files_processed += count
    pbar.update(count)

print(f"Processed {files_processed} files.")
pbar.close()

# 2 - Assign labels

In [ ]:
from data_preprocessing import aggregate_labels

label_csv = os.path.join(paths.DATA_DIR, "openmic", "openmic-2018-aggregated-labels.csv")

df, DISTINCT_INSTRUMENTS = aggregate_labels(label_csv)

df

In [ ]:
from data_preprocessing import calculate_binary_labels

POSTIVE_LABEL_CONFIDENCE_THRESHOLD = 0.8

df = calculate_binary_labels(df, tqdm=tqdm, lambda_=POSTIVE_LABEL_CONFIDENCE_THRESHOLD)

df.to_csv( os.path.join(paths.DATA_DIR, "calculated_labels.csv") )

df

In [ ]:
from data_preprocessing import load_features

load_features(df, tqdm=tqdm)

In [ ]:
# Exclude rows with a significance_score of 0

# df = df[df["significance_score"] > 0]

In [ ]:
from data_preprocessing import split_data

train_df, test_df = split_data(df, split_ratio = 0.7)

In [ ]:
import matplotlib.pyplot as plt

# plot distribution of "significance_score" in the training set
plt.hist(train_df["significance_score"], bins=100)
plt.xlabel("Significance Score")
plt.ylabel("Count")
plt.title("Distribution of Significance Scores in Training Set")

plt.plot()
plt.show()

# plot distribution of "significance_score" in the test set
plt.hist(test_df["significance_score"], bins=100)
plt.xlabel("Significance Score")
plt.ylabel("Count")
plt.title("Distribution of Significance Scores in Test Set")

plt.plot()
plt.show()

In [ ]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# train_df.sort_values(by='significance_score', inplace=True)

X_train_raw = train_df['features'].to_numpy()
X_test_raw  = test_df['features'].to_numpy()

train_significance_scores = train_df['significance_score'].to_numpy()

X_train = [ np.array(x).flatten() for x in X_train_raw ]
X_test  = [ np.array(x).flatten() for x in X_test_raw ]

# Keyed by instrument: str
y_test_dict  = {}
y_train_dict = {}


for instrument in DISTINCT_INSTRUMENTS:
    y_test_dict[instrument]  = test_df[instrument].to_numpy()
    y_train_dict[instrument] = train_df[instrument].to_numpy()


print("Example of bias towards negative class in training set for voice:\n")
print( f'    * num_negative_labels: {len([ x for x in y_train_dict["voice"] if x == False])}' )
print( f'    * num_positive_labels: {len([ x for x in y_train_dict["voice"] if x == True])}\n' )

X_train_balanced_dict = {}
y_train_balanced_dict = {}
X_val_balanced_dict   = {}
y_val_balanced_dict   = {}

for instrument in DISTINCT_INSTRUMENTS:
    balancer = SMOTE(random_state=42)
    # balancer = RandomUnderSampler()

    X_train_resampled, y_train_resampled = balancer.fit_resample(X_train, y_train_dict[instrument])

    # Split the resampled data into train and validation sets
    X_train_balanced, X_val_balanced, y_train_balanced, y_val_balanced = train_test_split( X_train_resampled, y_train_resampled )

    X_train_balanced_dict[instrument] = X_train_balanced
    y_train_balanced_dict[instrument] = y_train_balanced
    X_val_balanced_dict[instrument]   = X_val_balanced
    y_val_balanced_dict[instrument]   = y_val_balanced


print("Example of balanced training set for voice:\n")
print( f'    * num_negative_labels: {len([ x for x in y_train_balanced_dict["voice"] if x == False])}' )
print( f'    * num_positive_labels: {len([ x for x in y_train_balanced_dict["voice"] if x == True])}' )


# 3 - Train SVM Classifier

In [ ]:
print("Dataset sizes after balancing:\n")

for instrument in DISTINCT_INSTRUMENTS:
    print(f"{instrument}:".ljust(20), f"Train: {len(y_train_balanced_dict[instrument])}".ljust(15), f"Val: {len(y_val_balanced_dict[instrument])}")

In [ ]:
from svm_classify import svm_fit_instrument

if not os.path.exists(paths.SVM_MODELS_DIR):
    os.makedirs(paths.SVM_MODELS_DIR)

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "accordion"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "banjo"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "bass"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "cello"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "clarinet"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "cymbals"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "drums"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "flute"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "guitar"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "mallet_percussion"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "mandolin"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "organ"
param_grid = {
    # 'C': np.linspace(0, 10, 10),
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "piano"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "saxophone"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "synthesizer"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "trombone"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "trumpet"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "ukulele"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "violin"
param_grid = {
    # 'C': np.linspace(10, 100, 10),
    # 'kernel': ['rbf', 'poly'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip SVM_SKIP_TRAINING

instrument = "voice"
param_grid = {
    # 'C': np.linspace(10, 20, 5),
    # # 'kernel': ['poly'],
    # 'kernel': ['rbf', 'sigmoid'],
    # 'gamma':  ['scale']
}
svm_fit_instrument( instrument,
                    param_grid,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
from mlp_classify import mlp_fit_instrument

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "accordion"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "banjo"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "bass"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "cello"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "clarinet"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "cymbals"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "drums"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "flute"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "guitar"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "mallet_percussion"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "mandolin"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "organ"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "piano"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "saxophone"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "synthesizer"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "trombone"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "trumpet"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "ukulele"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "violin"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )

In [ ]:
%%skip MLP_SKIP_TRAINING

instrument = "voice"

mlp_fit_instrument( instrument,
                    X_train_balanced_dict[instrument], y_train_balanced_dict[instrument],
                    X_val_balanced_dict[instrument],   y_val_balanced_dict[instrument],
                    X_test, y_test_dict[instrument] )